In [1]:
from benchmark_funs import *

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [49]:
import tqdm
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
def evaluate(opti, p_ind=1000):
    s = 0
    for fun in benchmark_functions:
        assert fun.larger_better == False # 最小化
        interval_len = fun.ub - fun.lb
        # X = nn.init.kaiming_uniform_(torch.empty(1000, fun.dimension))
        # X = nn.init.trunc_normal_(torch.empty(1000, fun.dimension), fun.lb+interval_len/2, interval_len/6, a=fun.lb, b=fun.ub)
        X = nn.init.kaiming_uniform_(torch.empty(p_ind, fun.dimension))
        X = X.to(device)
        X.requires_grad_(True)
        optimizer = opti([X])
        # box = tqdm.tqdm(range(1000), desc=f"正在优化{fun.name}")
        box = tqdm.tqdm(range(fun.get_budget()//p_ind), desc=f"正在优化{fun.name}")
        best_loss = 1e10
        patience = 0
        for i in box:
            # X[X<fun.lb] = fun.lb
            # X[X>fun.ub] = fun.ub
            loss = fun(X).min() - fun.optival
            # loss = fun(X).sum()
            loss_item = loss.detach().numpy().item()
            box.set_postfix(loss=loss_item)
            if loss_item<best_loss:
                patience = 0
                best_loss = loss_item
            else:
                patience +=1
                if patience>100:
                    break
            loss.backward()
            optimizer.step()
            if X.grad.norm()<1e-8:
                break
            optimizer.zero_grad()
            
        s+=loss.detach().numpy().item()
    return s


In [ ]:
opti = lambda p: optim.Adam(p, lr=0.1)
evaluate(opti, p_ind=10)

正在优化Rosenbrock:  15%|█▌        | 4622/30000 [00:15<01:24, 300.83it/s, loss=0.0948]  


KeyboardInterrupt: 

In [64]:
# opti = lambda p: optim.AdamW(p, lr=3e-1)
opti = lambda p: optim.RMSprop(p, lr=1e-2)
# evaluate(opti, p_ind=10)
evaluate(opti, p_ind=1000)

正在优化Michal: 100%|██████████| 20/20 [00:00<00:00, 758.59it/s, loss=0.843]


12525.051821757954

In [24]:
opti = lambda p: optim.SGD(p, lr=0.1)
evaluate(opti)

正在优化Michal: 100%|██████████| 1000/1000 [00:03<00:00, 294.12it/s, loss=1.37] 


In [19]:
opti = lambda p: optim.AdamW(p, lr=3e-4)
evaluate(opti)

正在优化Michal: 100%|██████████| 1000/1000 [00:03<00:00, 290.30it/s, loss=-1.8]
